In [12]:
import numpy as np 
import pandas as pd 
import os
print(os.listdir("../input"))

['validation', 'monkey_labels.txt', '10449_44567_upload_training.zip', '10449_44567_upload_validation.zip', 'training']


In [13]:
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import sklearn
import sys
import tensorflow as tf
import time

from tensorflow import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)

2.2.0
sys.version_info(major=3, minor=6, micro=9, releaselevel='final', serial=0)
matplotlib 3.2.1
numpy 1.18.5
pandas 1.0.4
sklearn 0.23.1
tensorflow 2.2.0
tensorflow.keras 2.3.0-tf


In [14]:
train_dir = "../input/training"
valid_dir = "../input/validation"
label_file = "../input/monkey_labels.txt"
print(os.path.exists(train_dir))
print(os.path.exists(valid_dir))
print(os.path.exists(label_file))

print(os.listdir(train_dir))
print(os.listdir(valid_dir))

True
True
True
['n6', 'n5', 'n2', 'n1', 'n3', 'n0', 'n8', 'n7', 'n9', 'n4']
['n6', 'n5', 'n2', 'n1', 'n3', 'n0', 'n8', 'n7', 'n9', 'n4']


In [15]:
labels = pd.read_csv(label_file, header=0)
print(labels)

   Label     Latin Name              Common Name                     \
0  n0         alouatta_palliata\t    mantled_howler                   
1  n1        erythrocebus_patas\t    patas_monkey                     
2  n2        cacajao_calvus\t        bald_uakari                      
3  n3        macaca_fuscata\t        japanese_macaque                 
4  n4       cebuella_pygmea\t        pygmy_marmoset                   
5  n5       cebus_capucinus\t        white_headed_capuchin            
6  n6       mico_argentatus\t        silvery_marmoset                 
7  n7      saimiri_sciureus\t        common_squirrel_monkey           
8  n8       aotus_nigriceps\t        black_headed_night_monkey        
9  n9       trachypithecus_johnii    nilgiri_langur                   

    Train Images    Validation Images  
0             131                  26  
1             139                  28  
2             137                  27  
3             152                  30  
4             131 

In [16]:
height = 128
width = 128
channels = 3
batch_size = 64
num_classes = 10

train_datagen = keras.preprocessing.image.ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 40,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
    fill_mode = 'nearest',
)
train_generator = train_datagen.flow_from_directory(train_dir,
                                                   target_size = (height, width),
                                                   batch_size = batch_size,
                                                   seed = 7,
                                                   shuffle = True,
                                                   class_mode = "categorical")
valid_datagen = keras.preprocessing.image.ImageDataGenerator(rescale = 1./255)
valid_generator = valid_datagen.flow_from_directory(valid_dir,
                                                    target_size = (height, width),
                                                    batch_size = batch_size,
                                                    seed = 7,
                                                    shuffle = False,
                                                    class_mode = "categorical")
train_num = train_generator.samples
valid_num = valid_generator.samples
print(train_num, valid_num)

Found 1098 images belonging to 10 classes.
Found 272 images belonging to 10 classes.
1098 272


In [17]:
for i in range(2):
    x, y = train_generator.next()
    print(x.shape, y.shape)
    print(y)

(64, 128, 128, 3) (64, 10)
[[0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0.

In [18]:
model = keras.models.Sequential([
    keras.layers.Conv2D(filters=32, kernel_size=3, padding='same',
                        activation='relu', input_shape=[width, height, channels]),
    keras.layers.Conv2D(filters=32, kernel_size=3, padding='same',
                        activation='relu'),
    keras.layers.MaxPool2D(pool_size=2),
    keras.layers.Conv2D(filters=64, kernel_size=3, padding='same',
                        activation='relu'),
    keras.layers.Conv2D(filters=64, kernel_size=3, padding='same',
                        activation='relu'),
    keras.layers.MaxPool2D(pool_size=2),
    keras.layers.Conv2D(filters=128, kernel_size=3, padding='same',
                        activation='relu'),
    keras.layers.Conv2D(filters=128, kernel_size=3, padding='same',
                        activation='relu'),
    keras.layers.MaxPool2D(pool_size=2),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(num_classes, activation='softmax'),
])

model.compile(loss="categorical_crossentropy",
              optimizer="adam", metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 128, 128, 32)      896       
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 128, 128, 32)      9248      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 64, 64, 64)        18496     
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 64, 64, 64)        36928     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 32, 32, 128)      

In [ ]:
epochs = 300
history = model.fit_generator(train_generator,
                              steps_per_epoch = train_num // batch_size,
                              epochs = epochs,
                              validation_data = valid_generator,
                              validation_steps = valid_num // batch_size)

Epoch 1/300
17/17 [==============================] - 47s 3s/step - loss: 2.2939 - accuracy: 0.1219 - val_loss: 2.2416 - val_accuracy: 0.2188
Epoch 2/300
17/17 [==============================] - 47s 3s/step - loss: 2.1129 - accuracy: 0.2176 - val_loss: 1.9551 - val_accuracy: 0.2383
Epoch 3/300
17/17 [==============================] - 46s 3s/step - loss: 1.9150 - accuracy: 0.2979 - val_loss: 2.0283 - val_accuracy: 0.3242
Epoch 4/300
17/17 [==============================] - 46s 3s/step - loss: 1.8138 - accuracy: 0.3104 - val_loss: 1.6677 - val_accuracy: 0.3789
Epoch 5/300
17/17 [==============================] - 47s 3s/step - loss: 1.7226 - accuracy: 0.3569 - val_loss: 2.0488 - val_accuracy: 0.2383
Epoch 6/300
17/17 [==============================] - 48s 3s/step - loss: 1.7916 - accuracy: 0.3598 - val_loss: 1.5886 - val_accuracy: 0.4219
Epoch 7/300
17/17 [==============================] - 45s 3s/step - loss: 1.6278 - accuracy: 0.4072 - val_loss: 1.7636 - val_accuracy: 0.3516
Epoch 8/300
1

Epoch 59/300
17/17 [==============================] - 49s 3s/step - loss: 0.7534 - accuracy: 0.7331 - val_loss: 0.8769 - val_accuracy: 0.6953
Epoch 60/300
17/17 [==============================] - 47s 3s/step - loss: 0.6198 - accuracy: 0.7776 - val_loss: 0.7883 - val_accuracy: 0.7383
Epoch 61/300
17/17 [==============================] - 46s 3s/step - loss: 0.5355 - accuracy: 0.8104 - val_loss: 0.8266 - val_accuracy: 0.7578
Epoch 62/300
17/17 [==============================] - 48s 3s/step - loss: 0.5027 - accuracy: 0.8211 - val_loss: 1.0427 - val_accuracy: 0.6797
Epoch 63/300
17/17 [==============================] - 48s 3s/step - loss: 0.5627 - accuracy: 0.7979 - val_loss: 0.8648 - val_accuracy: 0.7500
Epoch 64/300
17/17 [==============================] - 47s 3s/step - loss: 0.5071 - accuracy: 0.8153 - val_loss: 0.8497 - val_accuracy: 0.7539
Epoch 65/300
17/17 [==============================] - 47s 3s/step - loss: 0.4859 - accuracy: 0.8211 - val_loss: 0.8588 - val_accuracy: 0.7578
Epoch 

17/17 [==============================] - 47s 3s/step - loss: 0.1399 - accuracy: 0.9526 - val_loss: 1.1499 - val_accuracy: 0.8203
Epoch 174/300
17/17 [==============================] - 47s 3s/step - loss: 0.1249 - accuracy: 0.9613 - val_loss: 0.8099 - val_accuracy: 0.8242
Epoch 175/300
17/17 [==============================] - 48s 3s/step - loss: 0.0817 - accuracy: 0.9724 - val_loss: 0.8675 - val_accuracy: 0.8281
Epoch 176/300
17/17 [==============================] - 50s 3s/step - loss: 0.1040 - accuracy: 0.9681 - val_loss: 0.7827 - val_accuracy: 0.8438
Epoch 177/300
17/17 [==============================] - 47s 3s/step - loss: 0.1428 - accuracy: 0.9584 - val_loss: 0.7144 - val_accuracy: 0.8594
Epoch 178/300
17/17 [==============================] - 48s 3s/step - loss: 0.1286 - accuracy: 0.9586 - val_loss: 0.7377 - val_accuracy: 0.8438
Epoch 179/300
17/17 [==============================] - 47s 3s/step - loss: 0.1141 - accuracy: 0.9691 - val_loss: 0.9451 - val_accuracy: 0.8203
Epoch 180/300

In [ ]:
def plot_learning_curves(history, label, epcohs, min_value, max_value):
    data = {}
    data[label] = history.history[label]
    data['val_'+label] = history.history['val_'+label]
    pd.DataFrame(data).plot(figsize=(8, 5))
    plt.grid(True)
    plt.axis([0, epochs, min_value, max_value])
    plt.show()
plot_learning_curves(history, 'accuracy', epochs, 0, 1)
plot_learning_curves(history, 'loss', epochs, 1.5, 2.5)